In [ ]:
        try:
            from .run_cell import run_cell
        except:
            from run_cell import run_cell
        from mistune import Markdown, Renderer
        from contextlib import contextmanager

In [ ]:
        def indents(code):
            if code:
                str = list(filter(lambda s: s.strip(), code.splitlines()))
                if str:  
                    return len(str[-1])-len(str[-1].lstrip())
            return 0

In [ ]:
        class ExtractCode(Renderer):
            code = """"""
            def block_code(self, code, lang=None):
                self.code += code + '\n'
                return super().block_code(code, lang)

            def codespan(self, code):
                self.code += ' '*indents(self.code) + code + '\n'
                return super().codespan(code)

        class CodeOnly(Markdown):
            def render(self, text, **kwargs):
                return setattr(self.renderer, 'code', """""") or super().render(text, **kwargs) and self.renderer.code

In [ ]:
        literate = CodeOnly(renderer=ExtractCode(), parse_inline_html=True, parse_block_html=True)

In [ ]:
        @contextmanager
        def run_literate(text):
            from IPython.display import display, Markdown
            display(Markdown(text))
            yield literate.render(text)

In [ ]:
        load_ipython_extension = run_cell(run_literate)